In [1]:
import sys
#sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')
sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
from common import helpers
from common.tests import partial_diff_test

from common import polynom_factory
import numpy as np
from itertools import product
from grid import cell, grid_layer, grid_manager
from grid.allocator_interface import InterfaceDofAllocator2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D

In [ ]:
sizes = [(0.,float(2**i)) for i in range(1)]

for size in sizes:
    mass_matrix = polynom_factory.local_funcfunc_matrix(order=3, dim=2, distribution='globatto', size=size)
    gradgrad = polynom_factory.local_gradgrad_matrix(order=3, distribution='globatto', dim=2, size=size)
    #gradfunc = polynom_factory.local_gradfunc_matrix(order=3, distribution='globatto', dim=2, size=size)[1][0]
    print(size)
    for (point,root) in gradgrad[1].items():
        print(point)
        partial_diff_test(matrix=gradgrad[0], point=point, pointnum=root, pointdict=gradgrad[1], mass_matrix=None)

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')

In [6]:
gm.grid_layers = [layer1, layer2]

In [7]:
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [8]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()

In [9]:
s2b = np.array([[ 1.22829849,  0.48464847, -0.25684608, -0.45610087],
       [ 0.23913821,  0.65966867,  0.1235421 , -0.02234897],
       [-0.1275535 ,  1.03266901,  0.15410032, -0.05921584],
       [-0.03430599,  1.34046152,  0.11031793, -0.41647347],
       [ 0.08818493, -0.06115703,  0.97521681, -0.00224471],
       [ 0.04581709, -0.324008  ,  0.58342807,  0.69476284],
       [-0.38762016,  0.29856527,  0.85477372,  0.23428118]])

b2s = np.array([[ 0.61414924,  0.59784553, -0.31888374, -0.03430599,  0.22046232,
         0.11454272, -0.19381008],
       [ 0.04846485,  0.32983433,  0.51633451,  0.2680923 , -0.03057852,
        -0.162004  ,  0.02985653],
       [-0.02568461,  0.06177105,  0.07705016,  0.02206359,  0.48760841,
         0.29171404,  0.08547737],
       [-0.22805044, -0.05587244, -0.14803959, -0.41647347, -0.00561177,
         1.73690711,  0.11714059]])

In [10]:
ma.get_weakly_connected_edges(cell=cell1)

[(((1, 0), (1, 1)),
  ((1, 0), (1, 2)),
  (3, 'lobatto'),
  ((0, 37), (1, 38), (2, 39), (3, 30)))]

In [11]:
ma._get_flat_list_of_ddofs(cell=cell1, edge=ma.get_weakly_connected_edges(cell=cell1)[0][0])

[(12, 9), (13, 10), (14, 11), (15, 3)]

In [12]:
b2s.T[0:4]

array([[ 0.61414924,  0.04846485, -0.02568461, -0.22805044],
       [ 0.59784553,  0.32983433,  0.06177105, -0.05587244],
       [-0.31888374,  0.51633451,  0.07705016, -0.14803959],
       [-0.03430599,  0.2680923 ,  0.02206359, -0.41647347]])

In [13]:
from scipy.sparse import coo_matrix,csr_matrix

In [15]:
gradgrad = polynom_factory.local_gradgrad_matrix(order=3, distribution='globatto', dim=2, size=(0,1))

In [16]:
gradgrad

(array([[  6.19047619e-01,  -2.33105738e-01,  -6.45133425e-02,
           3.96825448e-02,  -2.33105738e-01,  -2.59725576e-01,
           1.48809544e-01,  -6.25130512e-02,  -6.45133425e-02,
           1.48809544e-01,  -3.78934550e-02,   1.29098753e-02,
           3.96825448e-02,  -6.25130512e-02,   1.29098753e-02,
          -3.96825310e-03],
        [ -2.33105738e-01,   2.14285712e+00,  -3.96824312e-02,
          -6.45133167e-02,  -2.59725576e-01,  -1.52046012e+00,
          -4.01298182e-01,   1.48809518e-01,   1.48809544e-01,
           3.23647967e-02,   1.53282258e-01,  -3.78934519e-02,
          -6.25130512e-02,   3.96825542e-02,  -5.95238074e-02,
           1.29098747e-02],
        [ -6.45133425e-02,  -3.96824312e-02,   2.14285713e+00,
          -2.33105838e-01,   1.48809544e-01,  -4.01298182e-01,
          -1.52046006e+00,  -2.59725520e-01,  -3.78934550e-02,
           1.53282258e-01,   3.23647763e-02,   1.48809548e-01,
           1.29098753e-02,  -5.95238074e-02,   3.96825621e-02,

In [17]:
ma._weak_edge_connections

{((0, 0),
  ((1, 0), (1, 1))): {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 30)]},
 ((0, 1),
  ((1, 1), (1, 2))): {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 30)]},
 ((1, 0),
  ((1, 0), (1, 2))): {((0, 0), ((1, 0), (1, 1))): [(12, 9),
   (13, 10),
   (14, 11),
   (15, 3)], ((0, 1), ((1, 1), (1, 2))): [(12, 3),
   (13, 22),
   (14, 23),
   (15, 21)]}}